In [0]:
%sql
CREATE TABLE IF NOT EXISTS odace.map.silver_logement (
  --id_maille STRING COMMENT 'Identification de la maille utilisée pour l’estimation de l’indicateur',
  code_commune STRING COMMENT 'Code INSEE de la commune',
  lib_commune STRING COMMENT 'Nom de la commune',
  lib_epci STRING COMMENT 'Siren de l’EPCI',
  lib_dep STRING COMMENT 'Code du département',
  lib_reg STRING COMMENT 'Code de la région',
  prix_loyer STRING COMMENT 'Indicateur de loyer en €/m2',
  borne_inf_pred STRING COMMENT 'borne basse de l’intervalle de prédiction à 95% (€/m2)',
  borne_sup_pred STRING COMMENT 'borne supérieure de l’intervalle de prédiction à 95% (€/m2)',
  niveau_pred STRING COMMENT 'Niveau de la prédiction',
  --nombre_obs_com STRING COMMENT 'Nombre d’observations dans la commune',
  --nombre_obs_maille STRING COMMENT 'Nombre d’observations dans la maille',
  --coef_determination STRING COMMENT 'Coefficient de détermination ajusté du modèle hédonique servant à l’estimation de l’indicateur de loyer',
  rescued_data STRING
)
USING DELTA;

In [0]:
%sql

CREATE OR REPLACE TEMP VIEW bronze_merged AS 
SELECT *, 
  lpad(CASE WHEN INSEE_C IS NULL THEN INSEE ELSE INSEE_C END, 5, '0') AS INSEE_C_MERGED  
FROM odace.map.bronze_logement;

MERGE INTO odace.map.silver_logement AS silver
USING (
  SELECT * FROM (
    SELECT
      --id_zone AS id_maille,
      CASE
        WHEN INSEE_C_MERGED LIKE '132%' THEN '13055'
        WHEN INSEE_C_MERGED LIKE '693%' THEN '69123'
        WHEN INSEE_C_MERGED LIKE '751%' THEN '75056'
        ELSE INSEE_C_MERGED
      END AS code_commune,
      LIBGEO AS lib_commune,
      EPCI AS lib_epci,
      DEP AS lib_dep,
      REG AS lib_reg,
      loypredm2 AS prix_loyer,
      `lwr.IPm2` AS borne_inf_pred,
      `upr.IPm2` AS borne_sup_pred,
      TYPPRED AS niveau_pred,
      --nbobs_com AS nombre_obs_com,
      --nbobs_mail AS nombre_obs_maille,
      --R2_adj AS coef_determination,
      _rescued_data AS rescued_data,
      ingestion_timestamp,
      ROW_NUMBER() OVER (
        PARTITION BY CASE
          WHEN INSEE_C_MERGED LIKE '132%' THEN '13055'
          WHEN INSEE_C_MERGED LIKE '693%' THEN '69300'
          WHEN INSEE_C_MERGED LIKE '751%' THEN '75056'
          ELSE INSEE_C_MERGED
        END 
        ORDER BY ingestion_timestamp DESC
      ) AS rn
    FROM bronze_merged
  ) latest
  WHERE rn = 1
) AS bronze
ON silver.code_commune = bronze.code_commune
WHEN MATCHED THEN UPDATE SET
  --id_maille = bronze.id_maille,
  lib_commune = bronze.lib_commune,
  lib_epci = bronze.lib_epci,
  lib_dep = bronze.lib_dep,
  lib_reg = bronze.lib_reg,
  prix_loyer = bronze.prix_loyer,
  borne_inf_pred = bronze.borne_inf_pred,
  borne_sup_pred = bronze.borne_sup_pred,
  niveau_pred = bronze.niveau_pred,
  --nombre_obs_com = bronze.nombre_obs_com,
  --nombre_obs_maille = bronze.nombre_obs_maille,
  --coef_determination = bronze.coef_determination,
  rescued_data = bronze.rescued_data
WHEN NOT MATCHED THEN INSERT (
  --id_maille, 
  code_commune, 
  lib_commune, 
  lib_epci, 
  lib_dep, 
  lib_reg, 
  prix_loyer, 
  borne_inf_pred, 
  borne_sup_pred, 
  niveau_pred, 
  --nombre_obs_com, 
  --nombre_obs_maille, 
  --coef_determination, 
  rescued_data
) VALUES (
  --bronze.id_maille, 
  bronze.code_commune, 
  bronze.lib_commune, 
  bronze.lib_epci, 
  bronze.lib_dep, 
  bronze.lib_reg, 
  bronze.prix_loyer, 
  bronze.borne_inf_pred, 
  bronze.borne_sup_pred, 
  bronze.niveau_pred, 
  --bronze.nombre_obs_com, 
  --bronze.nombre_obs_maille, 
  --bronze.coef_determination, 
  bronze.rescued_data
);